In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, Binarizer

from sklearn.pipeline import Pipeline

from feature_engine.imputation import(
    AddMissingIndicator,
    MeanMedianImputer,
    CategoricalImputer
)

from feature_engine.encoding import (
    RareLabelEncoder,
    OrdinalEncoder
)

from feature_engine.transformation import LogTransformer

from feature_engine.selection import DropFeatures
from feature_engine.wrappers import SklearnTransformerWrapper

import joblib

from keras.models import Model
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.wrappers.scikit_learn import KerasRegressor
from keras.activations import relu, sigmoid
from keras.layers import Dense, Activation
from keras.models import Sequential

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.metrics import mean_squared_error

In [2]:
#import my_preprocessors as mypp #nuestra libreria

In [3]:
data = pd.read_csv('marketing_campaign.csv', sep='\t')
data

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,10870,1967,Graduation,Married,61223.0,0,1,13-06-2013,46,709,...,5,0,0,0,0,0,0,3,11,0
2236,4001,1946,PhD,Together,64014.0,2,1,10-06-2014,56,406,...,7,0,0,0,1,0,0,3,11,0
2237,7270,1981,Graduation,Divorced,56981.0,0,0,25-01-2014,91,908,...,6,0,1,0,0,0,0,3,11,0
2238,8235,1956,Master,Together,69245.0,0,1,24-01-2014,8,428,...,3,0,0,0,0,0,0,3,11,0


In [4]:
#data[['dia', 'mes', 'Year']] = data['Dt_Customer'].str.split('-', n=2, expand=True)

In [5]:
data.shape

(2240, 29)

In [6]:
#data["dia"] = pd.to_numeric(data["dia"])
#data["mes"] = pd.to_numeric(data["mes"])
#data["Year"] = pd.to_numeric(data["Year"])
del(data['Dt_Customer'])

In [7]:
data.shape

(2240, 28)

In [8]:
#type(data["dia"])

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
        data.drop(['NumWebPurchases'], axis=1),
        data['NumWebPurchases'],
        test_size=0.1,
        random_state=2022)

X_train.shape, X_test.shape

((2016, 27), (224, 27))

## Configuración del Machine Learning Pipeline

In [10]:
#Variables categoricas con NA
CATEGORICAL_VARS_WITH_NA_FREQUENT = []

#Variable categoricas con NA pero indicador de Missing
CATEGORICAL_VARS_WITH_NA_MISSING = []

#Variables numéricas con NA
NUMERICAL_VARS_WITH_NA = ['Income']

#Variables para binarización por sesgo fuerte
BINARIZE_VARS = ['MntFruits', 'MntSweetProducts', 'MntFishProducts', 'MntMeatProducts','MntWines','MntGoldProds']

#Variables categoricas a codificar sin ordinalidad
CATEGORICAL_VARS = ['Education', 'Marital_Status']

#Variables seleccionadas según análisis de Lasso
FEATURES = [
    'Education', 
    'Marital_Status',
    'ID',
    #'Year_Birth',
    'Income',
    'Kidhome',
    'Teenhome',
    'Recency',
    'MntWines',
    'MntFruits',
    'MntMeatProducts',
    'MntFishProducts',
    'MntSweetProducts',
    'MntGoldProds',
    'NumDealsPurchases',
    'NumCatalogPurchases',
    'NumStorePurchases',
    'NumWebVisitsMonth',
    'AcceptedCmp3',
    'AcceptedCmp4',
    'AcceptedCmp5',
    'AcceptedCmp1',
    'AcceptedCmp2',
    'Complain',
    'Z_CostContact',
    'Z_Revenue',
    'Response',
    #'dia',
    #'mes',
    #'Year',
]

In [11]:
#Selección de variables para entrenamiento
X_train = X_train[FEATURES]

In [12]:
#Seleccionamos variables para predicción
X_test = X_test[FEATURES]

In [13]:
#def create_model():
#    clasificador2 = Sequential()
#    clasificador2.add(Dense(input_dim=30, kernel_initializer="he_uniform", units=16, activation='relu'))
#    clasificador2.add(Dense(kernel_initializer="he_uniform", units=11, activation='relu'))
#    clasificador2.add(Dense(units=1, kernel_initializer="he_uniform", activation="linear"))
    # Parametros de entrenamiento.
#    clasificador2.compile(optimizer='adam', loss='mean_squared_error', metrics=['MSE'])

#    return clasificador2

# wrap the model using the function you created
#clf = KerasRegressor(build_fn=create_model,verbose=0)

#clasificador2.fit(X_train, y_train, batch_size=15, epochs=100)

## Machine Learning PipeLine

In [14]:
WebPurchase_pipeline = Pipeline([
    
    #============= IMPUTACIONES ===================#
    
    #1. Imputación de variables categoricas
    ('missing_imputation', 
         CategoricalImputer(imputation_method='missing', variables=CATEGORICAL_VARS_WITH_NA_MISSING)
    ),
    
    #2. Imputación de variables categoricas con NA basado en frecuencia.
    ('frequent_imputation', 
         CategoricalImputer(imputation_method='frequent', variables=CATEGORICAL_VARS_WITH_NA_FREQUENT)
    ),
    
    #3. Indicamos Faltante en variables numéricas para imputar
    ('missing_indicator', AddMissingIndicator(variables=NUMERICAL_VARS_WITH_NA)),
    
    #4. Imputación de mediana para variables categoricas
    ('mean_imputation', MeanMedianImputer(
        imputation_method='mean', variables=NUMERICAL_VARS_WITH_NA)
    ),
    
    #8. Binarización de Variables con Sesgo Fuerte
    ('binarizer', SklearnTransformerWrapper(
        transformer=Binarizer(threshold=0), variables=BINARIZE_VARS)
    ),
    
   
    ('rare_label_encoder', RareLabelEncoder(
        tol=0.01, n_categories=1, variables=CATEGORICAL_VARS)),
    
    ('categorical_encoder', OrdinalEncoder(
        encoding_method='ordered', variables=CATEGORICAL_VARS)),
    
    #=========== SCALER ==============
    ('scaler', MinMaxScaler()),
    
    #=========== ENTRENAMIENTO DEL MODELO ============
    ('Lasso',Lasso(alpha=0.01, random_state=2022)),

]) 

In [15]:
WebPurchase_pipeline.fit(X_train, y_train)

Pipeline(steps=[('missing_imputation', CategoricalImputer(variables=[])),
                ('frequent_imputation',
                 CategoricalImputer(imputation_method='frequent',
                                    variables=[])),
                ('missing_indicator',
                 AddMissingIndicator(variables=['Income'])),
                ('mean_imputation',
                 MeanMedianImputer(imputation_method='mean',
                                   variables=['Income'])),
                ('binarizer',
                 SklearnTransformerWrapper(tran...
                                           variables=['MntFruits',
                                                      'MntSweetProducts',
                                                      'MntFishProducts',
                                                      'MntMeatProducts',
                                                      'MntWines',
                                                      'MntGoldProds'])),
      

In [16]:
preds = WebPurchase_pipeline.predict(X_test)

In [17]:
preds

array([ 1.44958681,  2.29605933,  6.28160869,  2.32800468,  2.17342808,
        2.19546501,  2.38675605,  3.02934571,  4.56888995,  1.94420881,
        6.44794992,  1.84832928,  3.37980074,  6.04532369,  3.85250613,
        5.33912998,  6.07830598,  1.12446107,  6.01215782,  5.60513455,
        2.18463217,  3.00325878,  5.31062922,  5.34029073,  5.39141658,
        5.48471357,  4.96868484,  5.3203697 ,  6.52612409,  6.2158976 ,
        4.34144562,  4.19520118,  0.08014347,  6.39988689,  5.52932614,
        5.79441925,  2.85576973,  2.34772404,  7.83452719,  2.22121954,
        2.80338682,  2.42285369,  6.49587416,  4.04773689,  5.29453764,
        4.97975192,  1.43975646,  3.01004739,  5.5758958 ,  7.0729145 ,
        2.76162094,  5.03345968,  7.81073333,  3.93866851,  1.08250551,
        3.89192825,  2.10023996,  0.41026494,  1.41275202,  5.28706725,
        1.90782364,  6.29188692,  3.01004739,  1.8541946 ,  5.87807485,
        2.08289614,  7.08449185,  5.31351096,  1.64338949,  4.60

In [18]:
from sklearn.metrics import mean_squared_error 

In [19]:
mean_squared_error(np.exp(y_test), np.exp(preds), squared=False)

35548968385.08934

In [20]:
#Guardamos pipeline
joblib.dump(WebPurchase_pipeline, 'WebPurchase_pipeline.pkl')

['WebPurchase_pipeline.pkl']

In [21]:
#import joblib
#Guardamos pipeline, OJO modificar ruta para realizar esta parte sin inconvenientes.
#joblib.dump(r"C:\Users\kimbe\OneDrive - Universidad Galileo\Octavo Trimestre\Product Development\Proyecto 1 Kimberly Rivera\WebPurchase_pipeline", r"C:\Users\kimbe\OneDrive - Universidad Galileo\Octavo Trimestre\Product Development\Proyecto 1 Kimberly Rivera\WebPurchase_pipeline.pkl")

In [22]:
#type(WebPurchase_pipeline)